# Gerenciador de investimentos

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
# from lib import lib as l
from lib import plot as p
from lib import data as d
from lib import finance as f

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA', '#27FEFA'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2', '#162CF2'])

## Leitura dos dados

In [3]:
dinv, dport, dport_inv, dprices = f.read_invest()
dinvest = pd.concat([dinv, dport_inv]).reset_index(drop=True)

## Fluxos de investimentos

### Histórico de fluxos

In [4]:
dflow = dinvest.groupby('ID').apply(f.calc_flows)
dflow = dflow.reset_index(level=[0, 1])
dflow.pop('level_1')
dflow['Rend'] = dflow['Rendimento anual'].apply(d.pct2int)
dflow = dflow.sort_values(by=['Rend'], ascending=False).reset_index(drop=True)
a = dflow.pop('Rend')

### Fluxos ativos

In [5]:
dflow[dflow['Ativo']]

,ID,Valor inicial,Ganho,Meses,Rendimento,Rendimento mensal,Rendimento anual,Ganho em 1 ano,Ativo,Tipo
0,100.000,1308.000,313.000,5.607,23.93%,3.9%,58.08%,941.512,True,IVVB11
1,102.000,1390.300,230.700,4.197,16.59%,3.73%,54.93%,890.363,True,IVVB11
2,105.000,1478.000,143.000,2.557,9.68%,3.68%,54.06%,876.299,True,IVVB11
13,14.300,930.000,3.540,1.311,0.38%,0.29%,3.53%,32.931,True,Nu
17,13.000,10000.000,48.320,1.967,0.48%,0.25%,2.98%,299.018,True,Nu
18,15.000,3000.000,8.470,1.180,0.28%,0.24%,2.9%,87.238,True,Nu
22,16.000,3189.460,3.160,0.623,0.1%,0.16%,1.92%,61.317,True,Nu
32,17.000,3000.000,0.100,0.098,0.0%,0.03%,0.41%,12.192,True,Nu
35,1.400,8952.240,8.190,0.000,0.09%,0.0%,0%,0.000,True,Nu
36,109.000,1306.700,0.000,2.525,0.0%,0.0%,0.0%,0.000,True,Cash


## Performance

In [6]:
dnu = dflow[dflow['Meses'] > 0]
dnu = dnu[dnu['Tipo'] == 'Nu']

detf = dflow[dflow['Meses'] > 0]
detf = detf[detf['Tipo'] != 'Nu']

### Renda fixa

In [7]:
p.change_df_prop(f.calc_flows_overview(dnu), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
20183,307.53,0.27%,492.697,2.44%


### Renda Variável

In [8]:
p.change_df_prop(f.calc_flows_overview(detf), 28)

Valor base,Ganho atual,Rendimento atual,Ganho em 1 ano,Rendimento anual
10122.2,-988.2,-8.89%,-387.12,-3.82%


### Mensal

In [9]:
fig_prof = p.make_fig()
fig_prof_total = p.make_fig()

This is the format of your plot grid:
[ (1,1) x1,y1 ]

This is the format of your plot grid:
[ (1,1) x1,y1 ]



In [10]:
dret = d.pivot_agg(f.calc_flow_gains(dinvest[dinvest['Tipo'] != 'Cash']), val='ValorDelta', idx='Data', col='Tipo', agg={'ValorDelta':np.sum})
dret['Portfolio'] = dret.sum(axis=1)
cols = dret.columns[1:]
for c in cols:
    dret = d.window(dret, val=c)

In [11]:
palette1 = {'Portfolio': lcolors[0], 'BRAX11': lcolors[2], 'IVVB11': lcolors[0], 'BBSD11': lcolors[1], 'Cash': lcolors[4]}
palette2 = {'PortfolioAvg': dcolors[0], 'BRAX11Avg': dcolors[2], 'IVVB11Avg': dcolors[0], 'BBSD11Avg': dcolors[1], 'Cash': dcolors[4]}
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette1, y=dret.columns[5:6], plot_type='bar')
p.plot(dret, x='Data', fig=fig_prof_total, palette=palette2, y=dret.columns[10:11], name=['Portfolio médio'])

FigureWidget({
    'data': [{'hoverinfo': 'name+y',
              'marker': {'color': '#13E881'},
            …